# Scraping GitHub python files

This notebook will scrape GitHub for python files with any of the keywords specified.

To start, run the cell below:

In [46]:
%pip install requests
%pip install python-dotenv


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Specify search fields below:

In [47]:
search_items = ["scrape", "scraper", "request"]

Run the following cell to ensure the function is callable:

In [74]:
from dotenv import load_dotenv
import requests
import base64
import json
import time

load_dotenv()

os.getenv("GITHUB_ACCESS_TOKEN")

with requests.Session() as s:
    s.headers = {
        "Accept": "application/vnd.github+json", 
        "Authorization": "Bearer " + os.getenv("GITHUB_ACCESS_TOKEN"),
        "X-GitHub-Api-Version": "2022-11-28",
    }

    times = []
    
    def search(link, filename="out.txt"):
        print(f"Searching: {link}")
        res = s.get(link)
        next_link = res.headers.get("link")
        if res.json().get('items') is not None:
            for item in res.json().get('items'):
                if item.get('url') is not None:
                    res = s.get(item.get('url'))
                    data = res.json()
                    content = data.get("content")
                    if content is not None:
                        content = content.split('\n')
                        file_data = ""
                        for line in content:
                            line = base64.b64decode(line).decode("utf-8", "ignore")
                            idx = line.find("#")
                            file_data += line[0 : idx]
                        file_data += "\n---\nEND-OF-FILE\n---\n\n"
                        with open(filename, "a") as f:
                            f.write(file_data)
        times.append(time.time())
        if len(times) >= 9:
            if times[-1] - times[-9] <= 60:
                print("Waiting patiently for GitHub to delete its rate limit...")
                time.sleep(int(times[-1] - times[-9]) + 1)
        if next_link is not None:
            i_s = next_link.find('rel="prev", <')
            i_e = next_link.find('>; rel="next"')
            if i_e != -1:
                if i_s != -1:
                    next_link = next_link[i_s + len('rel="prev", <') : i_e]
                else:
                    next_link = next_link[1 : i_e]
                search(next_link, filename)

To run the program, run this final cell:

In [76]:
times = []
for item in search_items:
    filename = str(item) + ".txt"
    with open(filename, "w") as f:
        f.write("")
    search("https://api.github.com/search/code?q=" + str(item).replace(" ", "") + "+language:python", filename=filename)

Searching: https://api.github.com/search/code?q=scrape+language:python
Searching: https://api.github.com/search/code?q=scrape+language%3Apython&page=2
Searching: https://api.github.com/search/code?q=scrape+language%3Apython&page=3
Searching: https://api.github.com/search/code?q=scrape+language%3Apython&page=4
Searching: https://api.github.com/search/code?q=scrape+language%3Apython&page=5
Searching: https://api.github.com/search/code?q=scrape+language%3Apython&page=6
Searching: https://api.github.com/search/code?q=scrape+language%3Apython&page=7
Searching: https://api.github.com/search/code?q=scrape+language%3Apython&page=8
Searching: https://api.github.com/search/code?q=scrape+language%3Apython&page=9
Searching: https://api.github.com/search/code?q=scrape+language%3Apython&page=10
Searching: https://api.github.com/search/code?q=scrape+language%3Apython&page=11
Searching: https://api.github.com/search/code?q=scrape+language%3Apython&page=12
Searching: https://api.github.com/search/code?

KeyboardInterrupt: 